In [ ]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
import numpy as np

def ds_gen(N=50000, M=12):
    dat = []
    
    for i in range(N):
        tmp = []
        
        for _ in range(M):
            tmp.append(np.random.randint(7))
        
        dat.append([tmp, int(np.average(tmp))])
        
    return dat

tr, vl, ts = ds_gen(), ds_gen(), ds_gen()

print(tr[:4])

In [ ]:
import torch as T
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
class DS_PG(Dataset):
    def __init__(self, dat):
        self.dat = dat
    
    def __len__(self):
        return len(self.dat)
    
    def __getitem__(self, idx):
        inp = self.dat[idx][0]
        lbl = self.dat[idx][1]
        
        return np.asarray(inp), lbl

ld_tr = DataLoader(DS_PG(tr), batch_size=32, shuffle=True)
ld_vl = DataLoader(DS_PG(vl), batch_size=64)
ld_ts = DataLoader(DS_PG(ts), batch_size=64)

for inp, lbl in ld_tr:
    print(inp.shape, lbl.shape)
    
    break

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.mlp = nn.Sequential(*[nn.Linear(12, 32), nn.ReLU(), 
                                   nn.Linear(32, 8)])
        
    def forward(self, inp):
        out = self.mlp(inp)
        
        return out

model = Model().cuda()

out = model(inp.float().cuda())
print(out.shape)

In [ ]:
class Loss_PG(nn.Module):
    def __init__(self, M=4):
        super(Loss_PG, self).__init__()
        
        self.M = M
        
        self.bl = 0
        self.nb = 0
    
    def forward(self, out, lbl):
        lbl = lbl.cpu().numpy()
        lbl = np.repeat(lbl, self.M)
        
        out = nn.functional.log_softmax(out, dim=1)
        tmp = T.multinomial(T.exp(out), self.M, replacement=True).view((-1, ))
        tmp = tmp.data.cpu().numpy()
        out = T.cat([out[(i//self.M):(i//self.M)+1, tmp[i]] for i in range(out.shape[0]*self.M)], dim=0)
        
        rwd = [1 if c==True else -1 for c in (tmp==lbl)]
        
        ls = 0
        for i in range(tmp.shape[0]):
            ls += -out[i]*(rwd[i]-self.bl)
        ls /= tmp.shape[0]
        
        self.bl = (self.bl*self.nb + np.average(rwd))/(self.nb+1)
        self.nb += 1
        
        return ls

loss_func = Loss_PG(M=2).cuda()
optim = T.optim.Adam(model.parameters(), lr=0.0001)

EPOCHS = 200

In [ ]:
from tqdm import tqdm_notebook as tqdm

for e in tqdm(range(EPOCHS)):
    ls_ep = 0
    
    model.train()
    with tqdm(ld_tr) as TQ:
        for inp, lbl in TQ:
            out = model(inp.float().cuda())
            ls_bh = loss_func(out, lbl.cuda())
            
            optim.zero_grad()
            ls_bh.backward()
            optim.step()
            
            ls_bh = ls_bh.cpu().detach().numpy()
            TQ.set_postfix(ls_bh='%.3f'%(ls_bh))
            ls_ep += ls_bh
        
        ls_ep /= len(TQ)
        print('Ep %d: %.4f' % (e+1, ls_ep))
        
    acc_ep = 0
    
    model.eval()
    with tqdm(ld_vl) as TQ:
        for inp, lbl in TQ:
            out = model(inp.float().cuda())
            
            out = out.cpu().detach().numpy()
            out = np.argmax(out, axis=1)
            lbl = lbl.numpy()
            
            acc_bh = np.average(out==lbl)
            TQ.set_postfix(acc_bh='%.3f'%(acc_bh))
            acc_ep += acc_bh
        
        acc_ep /= len(TQ)
        print('%.4f'%(acc_ep))